In [1]:
import numpy as np
import pandas as pd
import random
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow import keras
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as k
import os

/home/nextg3/miniconda3/envs/fl/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-09 12:33:56.825558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 12:33:57.072272: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-09 12:33:57.110432: W tensorflow/compiler/xla/stream_executor/plat

In [2]:
def  sum_scaled_weights(scaled_weight_list):
    '''Returns the sum of the listed scaled weights.
    scaled average of the weights'''
    avg_grad = list()
    #  get the average grad over all client gradients
    for grad_list_tupel in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tupel, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


In [3]:
def scale_model_weights(weight,scalar):
    '''Function for scaling a model's weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar*weight[i])
    return weight_final

In [4]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model
lr = 0.01
coms_round = 100
loss = 'categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate = lr,
                momentum = 0.9
                )

In [43]:
# Read all the h5 files from the path
img_path = '/home/nextg3/Documents/Thesis/Code/FL/local_models'
# Get path list
image_paths = list(paths.list_files(img_path))
local_models  = []

# get all models
for p in image_paths:
    test_model = SimpleMLP()
    local_model = test_model.build(784,10)
    local_model.compile(loss=loss,
                        optimizer=optimizer,
                        metrics=metrics)
    # set weights of the global model as weights of the local modelk

    local_model.load_weights(p)
    scaled_model_weights = scale_model_weights(local_model.get_weights(),len(image_paths))
    local_models.append(scaled_model_weights)
    k.clear_session()



# Final weights
average_weights = sum_scaled_weights(local_models)
smlp_global = SimpleMLP()
global_model = smlp_global.build(784,10)
global_model.set_weights(average_weights)
global_model.save_weights('/home/nextg3/Documents/Thesis/Code/FL/global_model.h5')

# # update global model
# global_model.set_weights(average_weights)

In [30]:
# img_path = '/home/nextg3/Documents/Thesis/Code/FL/local_models'
# list(paths.list_files(img_path))
len(local_models)

10

In [27]:
test_model = SimpleMLP()
local_model = test_model.build(784,10)
local_model.compile(loss=loss,
                    optimizer=optimizer,
                    metrics=metrics)
# set weights of the global model as weights of the local modelk
local_model.load_weights('/home/nextg3/Documents/Thesis/Code/FL/weights.h5')

In [12]:
k.clear_session()

In [13]:
# Testing with pickle
import pickle

w = pickle.load(open('/home/nextg3/Documents/Thesis/Code/FL/weights.pkl','rb'))
test_model = SimpleMLP()
local_model = test_model.build(784,10)
local_model.compile(loss=loss,
                    optimizer=optimizer,
                    metrics=metrics)
# set weights of the global model as weights of the local modelk

local_model.set_weights(w)

In [23]:
weight_paths = list(paths.list_files('/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory'))
weight_paths = [f for f in weight_paths if ('.pkl' in f) or ('.h5' in f)]

In [24]:
weight_paths

['/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client2_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/global_model.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client9_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client6_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client8_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client0_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client5_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client3_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client1_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client7_weights.pkl',
 '/home/nextg3/Documents/Thesis/Code/FL/MNIST/Shared_memory/client4_weights.pkl']